In [ ]:
import pandas as pd
import os
from pandas.tseries.offsets import *
from math import sin, cos, sqrt, atan2, radians

# StationPlus
功能 在"日期"目录下生成"线路-station.txt"文件，上下行、当前站、下一站、站间距离、站间耗时
运算逻辑 同上
附加功能 通过1、3生成的文件计算站间距离（平均），过滤掉距离大于10公里，耗时1000秒以上的数据（/station/线路-dist.txt）

In [1]:
def distance(coord1, coord2):
    R = 6373.0
    lat1 = radians(coord1[0])
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d

def get_station(bus1,line):
    # bus1[(bus1['O_SPEED'] == 0) & (bus1['O_LONGITUDE'] != 0.0) & (bus1['O_LATITUDE'] != 0.0)]
    len = bus1.shape[0]
    d = 0
    for i in range(len-1):
        start_bus = bus1.iloc[i]
        time = start_bus['O_TIME']
        station = start_bus['O_NEXTSTATIONNO']
        on = start_bus['O_UP']
        loc = [start_bus['O_LATITUDE'], start_bus['O_LONGITUDE']]
        
        if i == 0:
            start_t = time
            start_loc = loc
        
        next_bus = bus1.iloc[i+1]
        next_loc = [next_bus['O_LATITUDE'], next_bus['O_LONGITUDE']]
        next_time = next_bus['O_TIME']
        next_station = next_bus['O_NEXTSTATIONNO']
        
        if (next_station == station):
            d = d + distance(loc, next_loc)
                
        else:
            if (next_station > station):
#                 s = str(bus1['O_UP'].iloc[i+1])+','+str(next_station-1)+','+str(
#                     bus1['O_LATITUDE'].iloc[i+1])+','+str(bus1['O_LONGITUDE'].iloc[i+1])+'\n'
                if (next_station == (station+1)):
                    d = d + distance(loc, next_loc)
                    diff_t = next_time - start_t
                    ss = str(bus1['O_UP'].iloc[i+1])+','+str(station-1)+','+str(
                        next_station-1)+','+str(d)+','+str(diff_t.seconds)+'\n'
                    with open('./'+day+'/'+str(line)+'-station.txt', 'a+') as fw2:
                        fw2.write(ss)
                    d = 0
                    start_t = next_time
#             elif (next_station < station):
#                 s = str(bus1['O_UP'].iloc[i+1])+','+str(next_station-1)+','+str(
#                     bus1['O_LATITUDE'].iloc[i])+','+str(bus1['O_LONGITUDE'].iloc[i])+'\n'
#             with open('./'+day+'/'+str(line)+'-test.txt', 'a+') as fw:
#                 fw.write(s)
        #print(on,station,' to ',next_station,' diff time ',diff.seconds)

        

for d in range(25, 32):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)

    train = pd.read_csv("./data/train201710"+day+".csv", sep=",")
    train['O_TIME'] = pd.to_datetime('2017-10-' + day + ' ' + train['O_TIME'], format='%Y-%m-%d %H:%M:%S')
    train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x: x.sort_values(by='O_TIME', ascending=True))
    
    if not os.path.exists('./'+day+'/'):
        os.mkdir('./'+day+'/')
    line = train['O_LINENO'].drop_duplicates()
    print(day,line.shape)
    for i in range(line.shape[0]):
        l = line.iloc[i]
        buses = train[train['O_LINENO'] == l]
#         print(l, buses.shape)
        get_station(buses, l)
    print(day, "pre finished")

FileNotFoundError: File b'/home/b418-xiwei/xzl/notebook/DC/data/train20171001.csv' does not exist

In [5]:
if not os.path.exists('./station/'):
    os.mkdir('./station/')
for i in range(line.shape[0]):
    num = line.iloc[i]
    bus = pd.DataFrame()
    station = pd.DataFrame()
    for d in range(1, 25):
        if d < 10:
            day = '0' + str(d)
        else:
            day = str(d)

        path = './' + str(day) + '/' + str(num) + '.txt';
        if os.path.exists(path):
            busx = pd.read_table(path, header=None, sep=' ', encoding='utf-8')
            bus = pd.concat([bus, busx])
        path = './' + str(day) + '/' + str(num) + '-station.txt';
        if os.path.exists(path):
            stationx = pd.read_table(path, header=None, sep=',', encoding='utf-8')
            station = pd.concat([station, stationx])

    if (bus.shape[0]>0 and station.shape[0]>0):
        # all days
        bus.columns = ["O_UP", "Station", "O_LATITUDE", "O_LONGITUDE"]
        bus = bus[(bus['O_LONGITUDE'] != 0.0) & (bus['O_LATITUDE'] != 0.0)]

        station.columns = ['O_UP', "From", "To", "Distance", 'O_TIME']
        station = station[(station['Distance'] < 10) & (station['O_TIME'] < 1000)]

        # bus
        stations = bus['Station'].drop_duplicates()
        for up in range(2):
            for s in range(1, stations.shape[0] + 1):
                station0 = station[(station['O_UP'] == up) & (station['From'] == s) & (station['To'] == (s + 1))]
                station0 = station0.sort_values(by='Distance', ascending=False)
                d = station0['Distance'].mean()
                s = str(up) + ',' + str(s) + ',' + str(s + 1) + ',' + str(d)
                with open('./station/' + str(num) + '-dist.txt', 'a+') as fw:  # with方式不需要再进行close
                    fw.write(s + '\n')
print("station finished")

station finished
